In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import csv, os, sys
import datetime
# necesitamos setear el path como variable de entorno para importar modulos
sys.path.append(os.path.join('..','..','libs'))
from read_smn import read_smn

In [2]:
lector = read_smn(os.path.join('..','..','Data','junio-SMN','horario'))
# Extraemos los datos de salta
tstamps, data = lector.filter_by_station('SALTA')
train_tstamps = tstamps[:600]
valid_tstamps = tstamps[600:]
train_temp = data[:600,0]
valid_temp = data[600:,0]